# 📝 2021년 데이터마이닝 기출문제 - 완벽 정리본
## ⭐ 시험장에서 바로 이해할 수 있도록 주석 완비!

### 문제 구성
- **문제 1**: 회귀분석 (Ford Focus 중고차 가격 예측) - 42점
- **문제 2**: 분류 (뇌경색 예측 - 의사결정트리, 랜덤포레스트) - 30점

# 문제 1. 회귀분석 (Ford Focus 중고차 가격) - 42점

## 🎯 목표: Ford Focus 중고차 가격(price) 예측
- **데이터**: focus-manual-petrol.csv
- **목표변수**: price (중고차 가격)
- **모델**: Linear Regression, Lasso, Ridge, ElasticNet
- **핵심 평가지표**: MAE, RMSE

## 📊 변수 설명
| 변수 | 설명 | 타입 | 비고 |
|------|------|------|------|
| model | 자동차 모델 | 범주형 | Focus만 있음 (제거) |
| year | 생산 년도 | 수치 | |
| price | 중고차 가격 | 수치 | 목표변수 |
| transmission | 변속기 형태 | 범주형 | Manual만 있음 (제거) |
| mileage | 주행거리 (mile) | 수치 | |
| fuelType | 유종 | 범주형 | Petrol만 있음 (제거) |
| engineSize | 엔진 크기 | 수치 | |

## 라이브러리 불러오기

In [ ]:
import pandas as pd
import numpy as np

# 회귀 모델
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# 분류 모델 (문제 2용)
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, classification_report

import matplotlib.pyplot as plt
import seaborn as sns

# dmba 라이브러리 (선택사항)
try:
    from dmba import regressionSummary, classificationSummary
    HAS_DMBA = True
    print("✅ dmba 라이브러리 사용 가능")
except:
    HAS_DMBA = False
    print("⚠️ dmba 없음 - sklearn metrics만 사용")

## 데이터 로드 및 탐색

In [ ]:
# ========================================
# 데이터 로드
# ========================================

df = pd.read_csv('focus-manual-petrol.csv')  # ← ⭐ 파일명 확인!

print("="*60)
print("📊 Ford Focus 데이터 정보")
print("="*60)
df.info()

print("\n" + "="*60)
print("🔍 결측치 확인")
print("="*60)
print(df.isnull().sum())

print("\n" + "="*60)
print("👀 데이터 샘플")
print("="*60)
print(df.head())

print("\n" + "="*60)
print("📈 기본 통계")
print("="*60)
print(df.describe())

In [ ]:
# ========================================
# 고유값 확인 (값이 1개뿐인 변수 찾기)
# ========================================

print("="*60)
print("🔍 각 변수의 고유값 개수")
print("="*60)

for col in df.columns:
    n_unique = df[col].nunique()
    print(f"{col}: {n_unique}개")
    if n_unique <= 5:  # 값이 5개 이하면 출력
        print(f"  → 값: {df[col].unique()}")

print("\n💡 model, transmission, fuelType은 값이 1개뿐 → 제거!")

## 문제 1-(1) 데이터 전처리 (2022년 기출 스타일 가정)

### 📌 요구사항 (일반적인 패턴)
- 종속변수: price
- 독립변수: year, mileage, engineSize (수치형만)
- 값이 1개뿐인 변수 제거: model, transmission, fuelType

In [ ]:
# ========================================
# 독립변수/종속변수 분리
# ========================================

print("="*60)
print("🔧 데이터 전처리")
print("="*60)

# 목표변수
outcome = 'price'

# 의미 없는 변수 제거 (값이 1개뿐인 변수)
useless_cols = ['model', 'transmission', 'fuelType']

# 독립변수 선택
predictors = [col for col in df.columns 
             if col not in [outcome] + useless_cols]

X = df[predictors]
y = df[outcome]

print(f"\n독립변수: {predictors}")
print(f"종속변수: {outcome}")
print(f"\n제거된 변수: {useless_cols}")
print(f"\n데이터 크기: {X.shape}")
print(f"\n목표변수 기본 통계:")
print(y.describe())

## 문제 1-(2) 10겹 교차검증 (Linear Regression)

### 📌 요구사항 (일반적인 패턴)
- Linear Regression
- **10겹 교차검증**
- 평가 지표: **MAE 또는 RMSE**

In [ ]:
# ========================================
# Linear Regression 10겹 교차검증
# ========================================

print("="*60)
print("📊 Linear Regression (10겹 CV)")
print("="*60)

# 모델 생성
lr = LinearRegression()

# 10겹 교차검증 (MAE)
cv_scores_mae = cross_val_score(
    lr,
    X,
    y,
    cv=10,
    scoring='neg_mean_absolute_error',
    n_jobs=-1
)

mae_cv = -cv_scores_mae.mean()
std_cv_mae = cv_scores_mae.std()

# 10겹 교차검증 (RMSE)
cv_scores_rmse = cross_val_score(
    lr,
    X,
    y,
    cv=10,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1
)

rmse_cv = -cv_scores_rmse.mean()
std_cv_rmse = cv_scores_rmse.std()

print("\n" + "="*60)
print("📊 Linear Regression 결과")
print("="*60)
print(f"10-Fold CV MAE:  £{mae_cv:,.2f} (±£{std_cv_mae:,.2f})")
print(f"10-Fold CV RMSE: £{rmse_cv:,.2f} (±£{std_cv_rmse:,.2f})")
print("="*60)

## 문제 1-(3) Lasso 회귀분석

### 📌 요구사항 (2022년 스타일)
- **RandomizedSearchCV** 사용
- alpha 범위: `np.linspace(0.0001, 1000, num=500)`
- **200번 반복**
- 평가 지표: `neg_mean_absolute_error`
- 최적 alpha로 **10겹 CV** 측정

In [ ]:
# ========================================
# Lasso 하이퍼파라미터 튜닝
# ========================================

print("="*60)
print("🔧 Lasso alpha 튜닝 (RandomizedSearchCV)")
print("="*60)

# alpha 후보 정의
alpha_range = np.linspace(0.0001, 1000, num=500)

param_dist_lasso = {
    'alpha': alpha_range
}

print(f"\nalpha 범위: {alpha_range.min():.4f} ~ {alpha_range.max():.2f}")
print(f"후보 개수: {len(alpha_range)}개")
print(f"랜덤 시도 횟수: 200번")

# RandomizedSearchCV
random_search_lasso = RandomizedSearchCV(
    Lasso(max_iter=10000, random_state=42),
    param_dist_lasso,
    n_iter=200,
    cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

print("\n⏳ RandomizedSearchCV 진행 중...")
random_search_lasso.fit(X, y)

# 최적 alpha
best_alpha_lasso = random_search_lasso.best_params_['alpha']
best_mae_lasso = -random_search_lasso.best_score_

print("\n" + "="*60)
print("📊 Lasso 최적 alpha")
print("="*60)
print(f"최적 alpha: {best_alpha_lasso:.6f}  ← ⭐ 답안!")
print(f"CV MAE: £{best_mae_lasso:,.2f}")
print("="*60)

In [ ]:
# ========================================
# 최적 alpha로 10겹 교차검증
# ========================================

print("\n" + "="*60)
print("🔄 Lasso 10겹 교차검증 (최적 alpha)")
print("="*60)

# 최적 모델
lasso_best = Lasso(alpha=best_alpha_lasso, max_iter=10000, random_state=42)

# 10겹 교차검증 (MAE)
cv_scores_lasso_mae = cross_val_score(
    lasso_best, X, y, cv=10, scoring='neg_mean_absolute_error', n_jobs=-1
)
mae_cv_lasso = -cv_scores_lasso_mae.mean()

# 10겹 교차검증 (RMSE)
cv_scores_lasso_rmse = cross_val_score(
    lasso_best, X, y, cv=10, scoring='neg_root_mean_squared_error', n_jobs=-1
)
rmse_cv_lasso = -cv_scores_lasso_rmse.mean()

print("\n" + "="*60)
print("📊 Lasso 결과")
print("="*60)
print(f"최적 alpha: {best_alpha_lasso:.6f}")
print(f"10-Fold CV MAE:  £{mae_cv_lasso:,.2f}  ← ⭐ 답안!")
print(f"10-Fold CV RMSE: £{rmse_cv_lasso:,.2f}")
print("="*60)

## 문제 1-(4) Ridge 회귀분석

### 📌 요구사항
- Lasso와 동일한 방식
- Ridge 모델 사용

In [ ]:
# ========================================
# Ridge 하이퍼파라미터 튜닝
# ========================================

print("="*60)
print("🔧 Ridge alpha 튜닝 (RandomizedSearchCV)")
print("="*60)

param_dist_ridge = {
    'alpha': alpha_range
}

# RandomizedSearchCV
random_search_ridge = RandomizedSearchCV(
    Ridge(max_iter=10000, random_state=42),
    param_dist_ridge,
    n_iter=200,
    cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

print("\n⏳ RandomizedSearchCV 진행 중...")
random_search_ridge.fit(X, y)

# 최적 alpha
best_alpha_ridge = random_search_ridge.best_params_['alpha']
best_mae_ridge = -random_search_ridge.best_score_

print("\n" + "="*60)
print("📊 Ridge 최적 alpha")
print("="*60)
print(f"최적 alpha: {best_alpha_ridge:.6f}  ← ⭐ 답안!")
print(f"CV MAE: £{best_mae_ridge:,.2f}")
print("="*60)

In [ ]:
# ========================================
# 최적 alpha로 10겹 교차검증
# ========================================

print("\n" + "="*60)
print("🔄 Ridge 10겹 교차검증 (최적 alpha)")
print("="*60)

ridge_best = Ridge(alpha=best_alpha_ridge, max_iter=10000, random_state=42)

# 10겹 교차검증
cv_scores_ridge_mae = cross_val_score(
    ridge_best, X, y, cv=10, scoring='neg_mean_absolute_error', n_jobs=-1
)
mae_cv_ridge = -cv_scores_ridge_mae.mean()

cv_scores_ridge_rmse = cross_val_score(
    ridge_best, X, y, cv=10, scoring='neg_root_mean_squared_error', n_jobs=-1
)
rmse_cv_ridge = -cv_scores_ridge_rmse.mean()

print("\n" + "="*60)
print("📊 Ridge 결과")
print("="*60)
print(f"최적 alpha: {best_alpha_ridge:.6f}")
print(f"10-Fold CV MAE:  £{mae_cv_ridge:,.2f}  ← ⭐ 답안!")
print(f"10-Fold CV RMSE: £{rmse_cv_ridge:,.2f}")
print("="*60)

## 문제 1-(5) ElasticNet 회귀분석

### 📌 요구사항
- alpha: `np.linspace(0.0001, 1000, num=500)`
- l1_ratio: `np.linspace(0, 1, num=500)`
- **max_iter=100000**
- RandomizedSearchCV 200번

In [ ]:
# ========================================
# ElasticNet 하이퍼파라미터 튜닝
# ========================================

print("="*60)
print("🔧 ElasticNet alpha & l1_ratio 튜닝")
print("="*60)

alpha_range_elastic = np.linspace(0.0001, 1000, num=500)
l1_ratio_range = np.linspace(0, 1, num=500)

param_dist_elastic = {
    'alpha': alpha_range_elastic,
    'l1_ratio': l1_ratio_range
}

print(f"\nalpha 범위: {alpha_range_elastic.min():.4f} ~ {alpha_range_elastic.max():.2f}")
print(f"l1_ratio 범위: {l1_ratio_range.min():.1f} ~ {l1_ratio_range.max():.1f}")
print(f"max_iter: 100000")

# RandomizedSearchCV
random_search_elastic = RandomizedSearchCV(
    ElasticNet(max_iter=100000, random_state=42),
    param_dist_elastic,
    n_iter=200,
    cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

print("\n⏳ RandomizedSearchCV 진행 중...")
random_search_elastic.fit(X, y)

# 최적 파라미터
best_alpha_elastic = random_search_elastic.best_params_['alpha']
best_l1_ratio = random_search_elastic.best_params_['l1_ratio']
best_mae_elastic = -random_search_elastic.best_score_

print("\n" + "="*60)
print("📊 ElasticNet 최적 파라미터")
print("="*60)
print(f"최적 alpha: {best_alpha_elastic:.6f}  ← ⭐ 답안!")
print(f"최적 l1_ratio: {best_l1_ratio:.6f}  ← ⭐ 답안!")
print(f"CV MAE: £{best_mae_elastic:,.2f}")
print("="*60)

In [ ]:
# ========================================
# 최적 파라미터로 10겹 교차검증
# ========================================

print("\n" + "="*60)
print("🔄 ElasticNet 10겹 교차검증 (최적 파라미터)")
print("="*60)

elastic_best = ElasticNet(
    alpha=best_alpha_elastic,
    l1_ratio=best_l1_ratio,
    max_iter=100000,
    random_state=42
)

# 10겹 교차검증
cv_scores_elastic_mae = cross_val_score(
    elastic_best, X, y, cv=10, scoring='neg_mean_absolute_error', n_jobs=-1
)
mae_cv_elastic = -cv_scores_elastic_mae.mean()

cv_scores_elastic_rmse = cross_val_score(
    elastic_best, X, y, cv=10, scoring='neg_root_mean_squared_error', n_jobs=-1
)
rmse_cv_elastic = -cv_scores_elastic_rmse.mean()

print("\n" + "="*60)
print("📊 ElasticNet 결과")
print("="*60)
print(f"최적 alpha: {best_alpha_elastic:.6f}")
print(f"최적 l1_ratio: {best_l1_ratio:.6f}")
print(f"10-Fold CV MAE:  £{mae_cv_elastic:,.2f}  ← ⭐ 답안!")
print(f"10-Fold CV RMSE: £{rmse_cv_elastic:,.2f}")
print("="*60)

## 문제 1-(6) 모델 비교

### 📌 요구사항
- (2), (3), (4), (5)의 결과 비교
- Linear, Lasso, Ridge, ElasticNet 성과 비교

In [ ]:
# ========================================
# 모델 비교
# ========================================

print("="*60)
print("🏆 문제 1-(6) 결과: 모델 비교")
print("="*60)

# 결과 정리
comparison_df = pd.DataFrame({
    'Model': ['Linear Regression', 'Lasso', 'Ridge', 'ElasticNet'],
    '10-Fold CV MAE': [
        mae_cv,
        mae_cv_lasso,
        mae_cv_ridge,
        mae_cv_elastic
    ],
    '10-Fold CV RMSE': [
        rmse_cv,
        rmse_cv_lasso,
        rmse_cv_ridge,
        rmse_cv_elastic
    ]
})

# 순위 추가
comparison_df['Rank (MAE)'] = comparison_df['10-Fold CV MAE'].rank()

# 정렬
comparison_df = comparison_df.sort_values('10-Fold CV MAE')

print("\n", comparison_df.to_string(index=False))

# 최고 모델
best_model_name = comparison_df.iloc[0]['Model']
best_model_mae = comparison_df.iloc[0]['10-Fold CV MAE']

print(f"\n" + "="*60)
print(f"✅ 최고 성능 모델: {best_model_name}")
print(f"✅ MAE: £{best_model_mae:,.2f}")
print("="*60)

print("\n💡 해석:")
print("- MAE가 작을수록 좋은 모델")
print("- Lasso/Ridge/ElasticNet은 규제로 과적합 방지")
print("- 데이터에 따라 최적 모델이 달라질 수 있음")

In [ ]:
# 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# MAE 비교
bars1 = axes[0].bar(comparison_df['Model'], comparison_df['10-Fold CV MAE'])
bars1[0].set_color('green')
bars1[0].set_alpha(0.8)
axes[0].set_xlabel('Model', fontsize=12, fontweight='bold')
axes[0].set_ylabel('10-Fold CV MAE (£)', fontsize=12, fontweight='bold')
axes[0].set_title('Model Comparison - MAE', fontsize=14, fontweight='bold')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(axis='y', alpha=0.3)

# RMSE 비교
bars2 = axes[1].bar(comparison_df['Model'], comparison_df['10-Fold CV RMSE'])
bars2[0].set_color('green')
bars2[0].set_alpha(0.8)
axes[1].set_xlabel('Model', fontsize=12, fontweight='bold')
axes[1].set_ylabel('10-Fold CV RMSE (£)', fontsize=12, fontweight='bold')
axes[1].set_title('Model Comparison - RMSE', fontsize=14, fontweight='bold')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

---

# 문제 2. 분류 (뇌경색 예측) - 30점

## 🎯 목표: 뇌경색 발생 예측
- **데이터**: healthcare-dataset-stroke-data.csv
- **목표변수**: stroke (1: 발생, 0: 발생 안 함)
- **모델**: 의사결정트리, 랜덤포레스트
- **핵심 평가지표**: Accuracy, AUC (ROC AUC)

## 📊 변수 설명
| 변수 | 설명 | 타입 |
|------|------|------|
| id | 환자번호 | 수치 (제거) |
| gender | 성별 | 범주형 (Male/Female) |
| age | 나이 | 수치 |
| hypertension | 고혈압 여부 | 수치 (0/1) → 범주형 변환 |
| heart_disease | 심장질환 여부 | 수치 (0/1) → 범주형 변환 |
| ever_married | 결혼 여부 | 범주형 (Yes/No) |
| work_type | 직업 종류 | 범주형 |
| Residence_type | 거주 형태 | 범주형 (Urban/Rural) |
| avg_glucose_level | 평균 혈당 수준 | 수치 |
| bmi | 체질량지수 | 수치 |
| smoking_status | 흡연 상태 | 범주형 |
| stroke | 뇌경색 발생 | 목표변수 (0/1) |

## 문제 2-(1) 결측치 제거

### 📌 요구사항
- NA 값이 포함된 행 제거

In [ ]:
# ========================================
# 데이터 로드
# ========================================

df_stroke = pd.read_csv('healthcare-dataset-stroke-data.csv')  # ← ⭐ 파일명 확인!

print("="*60)
print("📊 뇌경색 데이터 정보")
print("="*60)
df_stroke.info()

print("\n" + "="*60)
print("🔍 결측치 확인")
print("="*60)
print(df_stroke.isnull().sum())

print("\n" + "="*60)
print("👀 데이터 샘플")
print("="*60)
print(df_stroke.head())

In [ ]:
# ========================================
# 2-(1) 결측치 제거
# ========================================

print("\n" + "="*60)
print("🔧 2-(1) 결측치 제거")
print("="*60)

print(f"결측치 제거 전: {df_stroke.shape}")

# NA 제거
df_stroke = df_stroke.dropna()

print(f"결측치 제거 후: {df_stroke.shape}")
print(f"\n제거된 행 수: {df_stroke.shape[0]}")
print("\n✅ 결측치 제거 완료!")

## 문제 2-(2) 데이터 전처리

### 📌 요구사항
- **범주형 변수이지만 수치로 표현된 경우 범주형으로 변환**
  - hypertension (0/1) → 범주형
  - heart_disease (0/1) → 범주형
- 독립변수/종속변수 분리 (id 제외)
- **원-핫 인코딩** 수행

In [ ]:
# ========================================
# 2-(2) 범주형 변환 및 변수 분리
# ========================================

print("="*60)
print("🔧 2-(2) 데이터 전처리")
print("="*60)

# 수치로 표현된 범주형 변수를 범주형으로 변환
print("\n범주형 변환 중...")
df_stroke['hypertension'] = df_stroke['hypertension'].astype('category')
df_stroke['heart_disease'] = df_stroke['heart_disease'].astype('category')

print("✅ hypertension: 범주형으로 변환")
print("✅ heart_disease: 범주형으로 변환")

# 목표변수
outcome_stroke = 'stroke'

# 독립변수 (id 제외)
predictors_stroke = [col for col in df_stroke.columns 
                    if col not in [outcome_stroke, 'id']]

X_stroke = df_stroke[predictors_stroke].copy()
y_stroke = df_stroke[outcome_stroke]

print(f"\n독립변수: {predictors_stroke}")
print(f"종속변수: {outcome_stroke}")
print(f"\n목표변수 분포:")
print(y_stroke.value_counts())
print(f"\n비율:")
print(y_stroke.value_counts(normalize=True))

In [ ]:
# ========================================
# 원-핫 인코딩
# ========================================

# 범주형 변수 찾기
categorical_stroke = X_stroke.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"\n범주형 변수: {categorical_stroke}")

# 원-핫 인코딩
if len(categorical_stroke) > 0:
    print(f"\n원-핫 인코딩 중...")
    X_stroke = pd.get_dummies(X_stroke, columns=categorical_stroke, drop_first=True)
    print(f"인코딩 후 변수 개수: {X_stroke.shape[1]}개")

print(f"\n최종 독립변수 목록 (처음 10개): {X_stroke.columns.tolist()[:10]}")
print(f"최종 데이터 크기: {X_stroke.shape}")

## 문제 2-(3) 의사결정트리 (기본)

### 📌 요구사항
- 데이터 분리: **8:2** (학습:테스트)
- **최대나무** (제약 없음)
- **criterion='entropy'** ← ⭐ 문제에서 지정!
- 테스트 성과: **Accuracy**
- **변수 중요도 시각화**
- **5겹 교차검증**: **AUC** ← ⭐

In [ ]:
# ========================================
# 2-(3) 데이터 분리 (8:2)
# ========================================

X_train_stroke, X_test_stroke, y_train_stroke, y_test_stroke = train_test_split(
    X_stroke, y_stroke,
    test_size=0.2,      # ← ⭐ 테스트 20%
    random_state=42,
    stratify=y_stroke
)

print("="*60)
print("✂️ 데이터 분리 (8:2)")
print("="*60)
print(f"학습 데이터: {X_train_stroke.shape} (80%)")
print(f"테스트 데이터: {X_test_stroke.shape} (20%)")
print(f"\n학습 데이터 클래스 분포:")
print(y_train_stroke.value_counts())

In [ ]:
# ========================================
# 의사결정트리 모델 생성 (최대나무)
# ========================================

print("\n" + "="*60)
print("🌳 의사결정트리 - 최대나무 (entropy)")
print("="*60)

# 모델 생성
tree_full = DecisionTreeClassifier(
    criterion='entropy',  # ← ⭐ 문제에서 지정!
    random_state=42
    # max_depth=None (기본값 - 최대나무)
)

# 학습
tree_full.fit(X_train_stroke, y_train_stroke)

print("✅ 모델 학습 완료!")
print(f"\n[나무 구조 정보]")
print(f"최대 깊이: {tree_full.get_depth()}")
print(f"리프 노드 수: {tree_full.get_n_leaves()}")
print(f"총 노드 수: {tree_full.tree_.node_count}")

In [ ]:
# ========================================
# 테스트 데이터 평가 (Accuracy)
# ========================================

# 예측
y_pred_test_stroke = tree_full.predict(X_test_stroke)

# Accuracy
acc_test = accuracy_score(y_test_stroke, y_pred_test_stroke)

print("\n" + "="*60)
print("📊 테스트 데이터 성과 (Accuracy)")
print("="*60)
print(f"Accuracy: {acc_test:.4f}  ← ⭐ 답안!")
print("="*60)

In [ ]:
# ========================================
# 변수 중요도 시각화 ⭐
# ========================================

# 변수 중요도 추출
importance_df = pd.DataFrame({
    'feature': X_stroke.columns,
    'importance': tree_full.feature_importances_
}).sort_values('importance', ascending=False)

print("\n" + "="*60)
print("📊 변수 중요도")
print("="*60)
print("\n[Top 10 중요 변수]")
print(importance_df.head(10).to_string(index=False))

# 가장 중요한 3개 변수 저장
top3_features = importance_df.head(3)['feature'].tolist()
print(f"\n💡 가장 중요한 3개 변수: {top3_features}")
print("="*60)

In [ ]:
# 시각화
plt.figure(figsize=(10, 8))
top_15 = importance_df.head(15)
plt.barh(range(len(top_15)), top_15['importance'])
plt.yticks(range(len(top_15)), top_15['feature'])
plt.xlabel('Importance', fontsize=12, fontweight='bold')
plt.title('Top 15 Feature Importances', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# ========================================
# 5겹 교차검증 (AUC) ⭐
# ========================================

print("\n" + "="*60)
print("🔄 5겹 교차검증 (AUC)")
print("="*60)

# 5겹 교차검증
cv_scores_auc = cross_val_score(
    tree_full,
    X_stroke,
    y_stroke,
    cv=5,           # ← ⭐ 5겹
    scoring='roc_auc',  # ← ⭐ AUC
    n_jobs=-1
)

auc_cv_mean = cv_scores_auc.mean()
auc_cv_std = cv_scores_auc.std()

print("\n" + "="*60)
print("📊 문제 2-(3) 결과: 의사결정트리 (최대나무)")
print("="*60)
print(f"테스트 Accuracy: {acc_test:.4f}")
print(f"5-Fold CV AUC: {auc_cv_mean:.4f} (±{auc_cv_std:.4f})  ← ⭐ 답안!")
print(f"\n각 Fold AUC: {cv_scores_auc}")
print("="*60)

## 문제 2-(4) 의사결정트리 (Top 3 변수만)

### 📌 요구사항
- **(3)에서 찾은 가장 중요한 3개 변수만 사용**
- **최대나무**
- **criterion='entropy'**
- **5겹 교차검증 AUC**

In [ ]:
# ========================================
# 2-(4) Top 3 변수로 의사결정트리
# ========================================

print("="*60)
print("🌳 의사결정트리 (Top 3 변수)")
print("="*60)

print(f"\n사용 변수: {top3_features}")

# Top 3 변수로 데이터 준비
X_stroke_top3 = X_stroke[top3_features]

# 모델 생성
tree_top3 = DecisionTreeClassifier(
    criterion='entropy',
    random_state=42
)

# 5겹 교차검증
cv_scores_top3 = cross_val_score(
    tree_top3,
    X_stroke_top3,
    y_stroke,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1
)

auc_cv_top3 = cv_scores_top3.mean()
auc_cv_std_top3 = cv_scores_top3.std()

print("\n" + "="*60)
print("📊 문제 2-(4) 결과: 의사결정트리 (Top 3)")
print("="*60)
print(f"5-Fold CV AUC: {auc_cv_top3:.4f} (±{auc_cv_std_top3:.4f})  ← ⭐ 답안!")
print(f"\n각 Fold AUC: {cv_scores_top3}")
print("="*60)

print("\n💡 비교:")
print(f"모든 변수 AUC: {auc_cv_mean:.4f}")
print(f"Top 3 변수 AUC: {auc_cv_top3:.4f}")
print(f"차이: {auc_cv_mean - auc_cv_top3:.4f}")

## 문제 2-(5) 랜덤포레스트

### 📌 요구사항
- (2)의 데이터 사용 (모든 변수)
- **n_estimators=500** ← ⭐ 나무 500개
- **max_leaf_nodes=16** ← ⭐
- **max_features='auto'** ← ⭐
- **max_samples=0.5** ← ⭐
- **bootstrap=True** (중복 허용) ← ⭐
- **5겹 교차검증 AUC**

In [ ]:
# ========================================
# 2-(5) 랜덤포레스트
# ========================================

print("="*60)
print("🌲 랜덤포레스트")
print("="*60)

# 모델 생성
rf = RandomForestClassifier(
    n_estimators=500,        # ← ⭐ 나무 500개
    max_leaf_nodes=16,       # ← ⭐ 최대 리프 노드 16개
    max_features='sqrt',     # ← ⭐ 'auto'는 'sqrt'와 동일
    max_samples=0.5,         # ← ⭐ 50% 샘플링
    bootstrap=True,          # ← ⭐ 중복 허용
    random_state=42,
    n_jobs=-1
)

print("\n설정 파라미터:")
print(f"- n_estimators: 500")
print(f"- max_leaf_nodes: 16")
print(f"- max_features: 'sqrt' (auto와 동일)")
print(f"- max_samples: 0.5")
print(f"- bootstrap: True (중복 허용)")

# 5겹 교차검증
print("\n⏳ 5겹 교차검증 진행 중...")
cv_scores_rf = cross_val_score(
    rf,
    X_stroke,
    y_stroke,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1
)

auc_cv_rf = cv_scores_rf.mean()
auc_cv_std_rf = cv_scores_rf.std()

print("\n" + "="*60)
print("📊 문제 2-(5) 결과: 랜덤포레스트")
print("="*60)
print(f"5-Fold CV AUC: {auc_cv_rf:.4f} (±{auc_cv_std_rf:.4f})  ← ⭐ 답안!")
print(f"\n각 Fold AUC: {cv_scores_rf}")
print("="*60)

## 문제 2 모델 비교 (추가)

In [ ]:
# ========================================
# 문제 2 모델 비교
# ========================================

print("="*60)
print("🏆 문제 2 모델 비교")
print("="*60)

# 결과 정리
comparison_df_2 = pd.DataFrame({
    'Model': [
        'Decision Tree (All)',
        'Decision Tree (Top 3)',
        'Random Forest'
    ],
    '5-Fold CV AUC': [
        auc_cv_mean,
        auc_cv_top3,
        auc_cv_rf
    ]
})

# 순위
comparison_df_2['Rank'] = comparison_df_2['5-Fold CV AUC'].rank(ascending=False)
comparison_df_2 = comparison_df_2.sort_values('5-Fold CV AUC', ascending=False)

print("\n", comparison_df_2.to_string(index=False))

# 최고 모델
best_model_2 = comparison_df_2.iloc[0]['Model']
best_auc_2 = comparison_df_2.iloc[0]['5-Fold CV AUC']

print(f"\n" + "="*60)
print(f"✅ 최고 성능 모델: {best_model_2}")
print(f"✅ AUC: {best_auc_2:.4f}")
print("="*60)

print("\n💡 해석:")
print("- AUC가 높을수록 좋은 모델")
print("- 랜덤포레스트는 여러 나무의 앙상블로 성능 향상")
print("- Top 3 변수만 사용하면 성능이 떨어질 수 있음")

In [ ]:
# 시각화
plt.figure(figsize=(10, 6))
bars = plt.bar(comparison_df_2['Model'], comparison_df_2['5-Fold CV AUC'])

# 최고 모델 강조
bars[0].set_color('green')
bars[0].set_alpha(0.8)

plt.xlabel('Model', fontsize=12, fontweight='bold')
plt.ylabel('5-Fold CV AUC', fontsize=12, fontweight='bold')
plt.title('Model Comparison - AUC', fontsize=14, fontweight='bold')
plt.ylim([0, 1])
plt.xticks(rotation=15, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

---

# 🎯 최종 정리 및 답안 요약

## 문제 1. 회귀분석 (Ford Focus 중고차 가격)

### (2) Linear Regression
- **10-Fold CV MAE**: 출력된 값 확인
- **10-Fold CV RMSE**: 출력된 값 확인

### (3) Lasso
- **최적 alpha**: 출력된 값 확인
- **10-Fold CV MAE**: 출력된 값 확인

### (4) Ridge
- **최적 alpha**: 출력된 값 확인
- **10-Fold CV MAE**: 출력된 값 확인

### (5) ElasticNet
- **최적 alpha**: 출력된 값 확인
- **최적 l1_ratio**: 출력된 값 확인
- **10-Fold CV MAE**: 출력된 값 확인

### (6) 모델 비교
- **답안**: 비교 테이블 및 최고 모델 확인

---

## 문제 2. 분류 (뇌경색 예측)

### (1) 결측치 제거
- **답안**: 제거된 행 수 확인

### (2) 데이터 전처리
- **범주형 변환**: hypertension, heart_disease
- **원-핫 인코딩**: 완료

### (3) 의사결정트리 (최대나무)
- **테스트 Accuracy**: 출력된 값 확인
- **Top 3 중요 변수**: 출력된 변수 확인
- **5-Fold CV AUC**: 출력된 값 확인 ← ⭐ 답안!

### (4) 의사결정트리 (Top 3 변수)
- **5-Fold CV AUC**: 출력된 값 확인 ← ⭐ 답안!

### (5) 랜덤포레스트
- **5-Fold CV AUC**: 출력된 값 확인 ← ⭐ 답안!

---

## 💡 시험 팁

1. **⭐ 표시** 찾기 → 문제 핵심!
2. **평가 지표 주의** → MAE, RMSE (회귀) / Accuracy, AUC (분류)
3. **교차검증 횟수** → 5겹 vs 10겹
4. **criterion='entropy'** → 의사결정트리에서 지정!
5. **랜덤포레스트 파라미터** → n_estimators, max_leaf_nodes 등 정확히!
6. **범주형 변환** → 0/1로 표현된 변수도 범주형으로!
7. **변수 중요도** → 시각화 후 Top N 사용